In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup as bs
import requests
import time
import pandas as pd
from datetime import datetime, timedelta

In [2]:
# 網頁共有幾頁
def get_page():
    page_source = bs(browser.page_source,'html.parser')
    page = page_source.find_all('center')[0]
    # 查找last是否存在
    if len(page.select('.p_last')):
        page = page.select('.p_last')[0]
        page = int(page['href'][-1])
    else :
        page = 1
        
    return page

# get 全部網址
def get_url():
    page = get_page()

    for p in range(page):
        soup = bs(browser.page_source,'html.parser')
        
        for ele in range(len(soup.select('#newslistul li a.tit'))):
            title.append(soup.select('#newslistul li a.tit')[ele].text.strip('\n'))
            url.append(soup.select('#newslistul li a.tit')[ele]['href'])
            time.sleep(3)

        # 若是爬完本頁title及網址，則按下一頁
        if p < page-1 :
            browser.find_element_by_class_name('p_next').click()
            time.sleep(3)
            
    return title, url

# get 全部新聞內文
## 利用網址進去網頁爬內文

def get_news(url):
    # 使用假 header
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36'}

    for u in range(len(url)):
        urls = 'https://news.ltn.com.tw/' + str(url[u])
        web_data = requests.get(urls, headers=headers)
        web_data.encoding = 'utf-8'
        soup = bs(web_data.text,'html.parser')

        # 標題
        data_title = soup.title.text
        all_title.append(data_title)
        time.sleep(1)

        # 日期
        if len(soup.select('.text span')):
            data_date = soup.select('.text span')[0].text
            data_date = data_date.strip()
            data_date = pd.to_datetime(data_date)
            data_date = data_date.date()
            date.append(data_date)

        elif len(soup.select('.c_time')):
            data_date = soup.select('.c_time')[0].text
            data_date = data_date.strip()
            data_date = pd.to_datetime(data_date)
            data_date = data_date.date()
            date.append(data_date)

        elif len(soup.select('.date')):
            data_date = soup.select('.date')[0].text
            data_date = data_date.strip()
            data_date = pd.to_datetime(data_date)
            data_date = data_date.date()
            date.append(data_date)

        elif len(soup.select('.writer_date')):
            data_date = soup.select('.writer_date')[0].text
            data_date = data_date.strip()
            data_date = pd.to_datetime(data_date)
            data_date = data_date.date()
            date.append(data_date)

        elif len(soup.select('.viewtime')):
            data_date = soup.select('.viewtime')[0].text
            data_date = data_date.strip()
            data_date = pd.to_datetime(data_date)
            data_date = data_date.date()
            date.append(data_date)
    
        else :
            print('date:',u)

        time.sleep(1)

        # 內文
        one_sentence = []

        if len(soup.select('.text p')):
            for i in range(len(soup.select('.text p'))):
                # 搜集"還想看更多新聞嗎？"前的所有文字
                if soup.select('.text p')[i].text != soup.select('.appE1121')[0].text :
                    one_sentence.append(soup.select('.text p')[i].text)
                else :
                    break

        elif len(soup.select('.cont p')):
            for i in range(len(soup.select('.cont p'))):
                # 搜集"還想看更多新聞嗎？"前的所有文字
                if soup.select('.cont p')[i].text != soup.select('.appE1121')[0].text :
                    one_sentence.append(soup.select('.cont p')[i].text)
                else :
                    break
                    
        elif len(soup.select('.news_content p')):
            for i in range(len(soup.select('.news_content p'))):
                # 搜集"還想看更多新聞嗎？"前的所有文字
                if soup.select('.news_content p')[i].text != soup.select('.appE1121')[0].text :
                    one_sentence.append(soup.select('.news_content p')[i].text)
                else :
                    break
                    
        elif len(soup.select('p')):
            s_sele = soup.select('p')[5:16]
            select_p = []
            for s in range(len(s_sele)):
                if (s % 2) == 0:
                    select_p.append(s_sele[s])

            for i in range(len(select_p)):
                one_sentence.append(select_p[i].text)
            
        else :
            print('content:',u)

        # 把同一篇的句子合併
        sentence = ''.join(one_sentence)
        content.append(sentence)
        time.sleep(3)

    return all_title, date, content

s_Year = ['2016','2017','2018']
s_Month = ['1', '4', '7', '9', '12']
e_Month = ['3', '6', '8', '11', '12']
e_Day = ['31', '30']

title = []
url = []

all_title = []
date = []
content = []

browser = webdriver.PhantomJS('C:/Users/joyle/Anaconda3/Scripts/phantomjs.exe')
# browser = webdriver.PhantomJS()
browser.get('https://news.ltn.com.tw/search?keyword=')
browser.find_element_by_id('keyword_search').send_keys("澳幣")

for i in range(len(s_Year)):
    # 選擇起始年份
    SYear = Select(browser.find_element_by_id('SYear'))
    SYear.select_by_value(s_Year[i])
    # 選擇結束年份
    EYear = Select(browser.find_element_by_id('EYear'))
    EYear.select_by_value(s_Year[i])
    for y in range(len(s_Month)):
        # 選擇起始月份
        SMonth = Select(browser.find_element_by_id('SMonth'))
        SMonth.select_by_value(s_Month[y])
        # 選擇起始日期
        SDay = Select(browser.find_element_by_id('SDay'))
        SDay.select_by_value("1")

        #選擇結束月份
        EMonth = Select(browser.find_element_by_id('EMonth'))
        EMonth.select_by_value(e_Month[y])
        if ( y % 2 ) == 0 :
            #選擇結束日期
            EDay = Select(browser.find_element_by_id('EDay'))
            EDay.select_by_value(e_Day[0])
            day = e_Day[0]
        else :
            EDay = Select(browser.find_element_by_id('EDay'))
            EDay.select_by_value(e_Day[1])
            day = e_Day[1]

       # print('起始日期:',s_Year[i], s_Month[y],'1','結束日期:',s_Year[i],e_Month[y],day)

        browser.find_element_by_xpath('/html/body/div[3]/section/div[3]/div[2]/form/input').click()

        time.sleep(3)

        title, url = get_url()

# browser.save_screenshot('news1.png') #螢幕截圖        
browser.quit() # 離開網頁

all_title, date, content = get_news(url)

date.sort()
all_title = pd.DataFrame(all_title)
date = pd.DataFrame(date)
content = pd.DataFrame(content)

all_title.columns = ['title']
date.columns = ['date']
content.columns = ['content']

df = pd.concat([date, all_title, content], axis = 1)
df.to_csv('news_2016-2018.csv',index=False, encoding='utf_8_sig')


/opt/python-3.6.8/lib/python3.6/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '
